In [2]:
from elasticsearch import Elasticsearch, helpers
import json

es = Elasticsearch('http://localhost:9200')

index_name = 'recipes'
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)

with open('recipes.json', 'r') as json_file:
    recipes = json.load(json_file)

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "ingredients": {"type": "text"},
            "steps": {"type": "text"},
            "name": {"type": "text"},
            "description": {"type": "text"},
            "tags": {"type": "text"},
            "n_ingredients": {"type": "text"},
            "n_steps": {"type": "text"},
            "id": {"type": "keyword"},
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "recipes"

es.indices.delete(index=index_name, ignore_unavailable=True)
es.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'recipes'})

In [11]:
from elasticsearch import Elasticsearch
from langchain.chains import ElasticsearchDatabaseChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
openai_api_key = os.getenv('OPEN_AI_API')

llm = OpenAI(api_key=openai_api_key, model_name='gpt-3.5-turbo')

es_chain = ElasticsearchDatabaseChain((
    database=es, 
    top_k = 5
)

prompt_template = PromptTemplate(
    input_variables=["context", "query"],
    template="""
    You are a helpful assistant who provides recipes based on user queries.
    Context: {context}
    User Query: {query}
    Please provide a detailed response based on the context and query.
    """
)

vector_store = ElasticsearchVectorStore(
    es_client=es_client,
    index_name=index_name,
    embedding_dim=384,
    embedding_field="text_vector",
    content_fields=["name", "description", "steps", "ingredients", "id"]
)


chain = RetrievalQA(
    retriever=vector_store,
    prompt_template=prompt_template,
    llm=llm
)

def rag(query):
    response = chain(query)
    return response['result']

# Example usage
query = "I have sausages, apples, and potatoes. What can I make?"
answer = rag(query)
print(answer)

SyntaxError: invalid syntax (3642384642.py, line 14)

In [19]:
from elasticsearch import Elasticsearch
from langchain.chains.elasticsearch_database.base import ElasticsearchDatabaseChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv('OPEN_AI_API')
llm = OpenAI(api_key=openai_api_key, model_name='gpt-3.5-turbo')

query_chain = LLMChain(
    llm=OpenAI(api_key="your_openai_api_key"),
    prompt_template="You are a helpful assistant. Answer the following question based on the context: {context} Question: {query}"
)

# Define your answer chain
answer_chain = LLMChain(
    llm=OpenAI(api_key="your_openai_api_key"),
    prompt_template="Generate an answer based on the retrieved documents: {documents} and the question: {query}"
)

# Configure ElasticsearchDatabaseChain
db_chain = ElasticsearchDatabaseChain(
    answer_chain=answer_chain,
    query_chain=query_chain,
    database=es,
    top_k=5,  # Number of top results to retrieve
    verbose=False  # Enable verbose mode for logging
)

inputs = {
    "context": "Information about recipes",
    "query": "How do I make a chocolate cake?"
}

response = db_chain(inputs)
print(response)

c:\Users\zzelk\anaconda3\envs\recipes\lib\site-packages\langchain_community\llms\openai.py:253: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


ValidationError: 1 validation error for OpenAIChat
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)